<img src="https://raw.githubusercontent.com/FUlyankin/r_probability/master/end_seminars/sem_6/student_teacher_dist.png" height="300" width="600"> 

# <center> R для тервера и матстата. <br>  <br> Домашка номер четыре </center>


Данный ноутбук является домашкой по курсу «R для теории вероятностей и математической статистики» (РАНХиГС, 2019). Автор ноутбука - [вот этот парень по имени Филипп.](https://vk.com/ppilif) Если у вас для него есть деньги, слава или женщины, он от этого всего не откажется. Ноутбук распространяется на условиях лицензии [Creative Commons Attribution-Share Alike 4.0.](https://creativecommons.org/licenses/by-sa/4.0/) При использовании обязательно упоминание автора курса и аффилиации. При наличии технической возможности необходимо также указать активную гиперссылку на [страницу курса.](https://fulyankin.github.io/R_probability/) На ней можно найти другие материалы. Фрагменты кода, включенные в этот notebook, публикуются как [общественное достояние.](https://creativecommons.org/publicdomain/zero/1.0/)

----------------------


Приветствую вас внутри четвёртой домашки. __Краткий брифинг:__

* __Дедлайн: 10.07.19__
* Эта домашка про проверку гипотез
* В каждой задачке нужно чётко обосновывать какой критерий вы используете, выписывать какую гипотезу вы проверяете, и какие предпосылки для этого должны быть выполнены. Если этого не сделано, буду снимать баллы. 
* На забывайте тестировать предпосылки. То есть это означает, что __если вы руководствуетесь предпосылкой о нормальности выборки, эту предпосылку надо проверить.__

---------------------------

* __Как оформлять домашки:__ 
    1. Компьютерные домашки оформляем либо в связке R + LaTeX, либо в юпитерских тетрадках (если хочется новых технологий), либо в связке R + Markdown. Все шаблоны для этого есть в репозитории. Пожалуйста, не нужно присылать мне просто тупой код с выводами в виде комментариев, написанных через решёточки. Я от этого могу сойти с ума. 
    2. Ручные решение можно оформлять двумя способами: либо вбивать их в файл вместе с кодом. Благо в Markdown, юпитере и других местах поддерживаются теховские формулу. Либо аккуратно оформляйте их на бумаге и присылайте свёрстанную pdf-ку. Не надо кидать мне фотки по одной. Я от этого тоже сойду с ума. Вариант с вбиванием в основной файл - предпочтительнее. Заодно попрактикуетесь в формулах для LaTeX. 
    
* На странице курса висит [видео-инструкция по оформлению.](https://yadi.sk/i/Pxp_pByP6Em9-A)

---------------------------

Ближе к делу. С чего начинается любой скрипт? Правильно! С подгрузки пакетов :) 

In [4]:
z.prop = function(m, n, p0, alpha=0.05, alternative="two-sided"){
  
  p_hat = m/n                       # оценка вероятности 
  sd_p = sqrt(p_hat*(1-p_hat)/n)    # дисперсия оценки
  z_stat = (p_hat - p0)/sd_p        # наблюдаемое значение

  # В зависимости от типа альтернативы делаем расчёты
  if(alternative == 'two-sided'){
      p_val <- 2*pnorm(-abs(z_stat))
      z_crit <- qnorm(1-alpha/2)
  }
  if(alternative == 'lower'){
      p_val <- pnorm(-abs(z_stat))
      z_crit <- qnorm(alpha)
  }
  if(alternative == 'greater'){
      p_val <- 1 - pnorm(abs(z_stat))
      z_crit <- qnorm(1-alpha/2)
  }
    
  # Возвращаем результаты проверки гипотезы в виде списка
  return( list(estimate = p_hat, 
               z_crit = z_crit,
               z_stat = z_stat,
               p_val = p_val,
               alternative = alternative))
}

# вот так можно к чему-нибудь применить функцию 
result <- z.prop(52, 100, 0.5, alpha=0.01, alternative = 'greater')

result  # вот так можно посмотреть на всё, что мы в неё запихнули

$estimate
[1] 0.52

$z_crit
[1] 2.575829

$z_stat
[1] 0.4003204

$p_val
[1] 0.3444603

$alternative
[1] "greater"

In [5]:
result$p_val # можно по именам из листа доставать разные циферки :3

[1] 0.3444603

Если кишка тонка писать свои функции, пользуйтесь готовыми. Но имейте в виду, что не все критерии, которые понадобятся вам в этой тетрадке, реализованы в R в виде одной команды. Если есть желание, можете снабдить выдачу картиночками как в лекции. Но только сделайте это покомпактнее. Также не путайте картинки местами! Например, не вздумайте использовать картинку для нормального распределения для t-критерия! Если код в ggplot кажется вам тяжёлым, можете попробовать рисовать картинки [как во втором ответе тут.](https://stackoverflow.com/questions/36508020/can-r-visualize-the-t-test-or-other-hypothesis-test-results)

# Рыбёшки 

### [ ] Задачка 1 

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало $50$ испытуемых. Каждый из них разговаривал с рекрутером; в конце $34$ из $50$ бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания $10$ из $34$ испытуемых экспериментальной группы и $4$ из $16$ испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно $4.4\%$. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости ($p-$значение) при альтернативе заразительности зевоты.

In [ ]:
# Ваш код

### [ ] Задачка 2

Ежегодно более $200000$ людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет $525$ баллов, стандартное отклонение — $100$ баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — $541.4$. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости $0.05$ нулевая гипотеза? 

In [ ]:
# Ваш код

### [ ] Задачка 3

Во время ЧЕ по футболу 2008 года и ЧМ 2010 года [Осьминог Пауль](https://ru.wikipedia.org/wiki/Пауль_(осьминог))занимался прогнозированием побед (после он ушёл в финансовую аналитику и IB). Осьминог дал верные прогнозы в $12$ случаях из $14$. Если предположить, что Пауль выбирает победителя наугад, какова вероятность получить 12 верных прогнозов из 14? 

In [ ]:
# НЕ НАДО НИЧЕГО ТУТ ГЕНЕРИРОВАТЬ! ПРОСТО ПОСЧИТАЙТЕ! 

Можно ли утверждать, что Осьминог обладает даром предвидения? Каким получилось p-значение? 

In [ ]:
# Ваш код

### [ ] Задачка 4

Новый корм для котиков рекламируется с помощью рекламного баннера. Отдел маркетинга разработал для рекламы новый баннер. Они считают, что он более красивый. Перед фирмой возникла необходимость проверить какой из двух баннеров лучше. 

Для этого была создана специальная веб-форма. В этой веб-форме людям по очереди демонстрировали оба баннера и просили поставить лайк, если баннер нравится человеку. Если человеку нравятся оба баннера, он может поставить два лайка. Если ему не нравится ни один баннер, он может не ставить лайки вовсе. Собранные в конечном итоге данные находятся в [табличке.](https://yadi.sk/i/9Z5N-6PM3XN8on)

Можно ли утверждать, что оба баннера одинаково хороши? Не забудьте найти p-значение.

In [ ]:
df = read.csv('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/banner_click_stat.csv', sep='\t', header = FALSE)
names(df) = c('banner_a', 'banner_b')
head(df)

In [ ]:
# Ваш код

Как думаете, какие у этого экперимента могли быть проблемы в плане его дизайна? 

In [ ]:
# Ваши мысли

### [ ] Задачка 5

Оборудование часто грешит и работает криво. Для каждого станка есть своя допустимая степень кривизны. Например, [в табличке](https://yadi.sk/i/bsjOHeOO3XaTN9) `gear.csv` лежат данные о производстве шестерёнок на станке. Первая колонка это диаметр, произведённой шестерни. Вторая колонка это то, к какой серии относится шестерня. 

Ясное дело, что станок не всегда делает диаметр равным $1$. У него есть некоторая погрешность. Будем считать, что станок годный, если дисперсия диаметра выпускаемых им шестерёнок не превышает $0.01$. Правда ли, что станок годный?

In [31]:
df = read.csv('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/gear.csv', sep=',')
head(df)

DIAMETER,BATCH
1.006,1
0.996,1
0.998,1
1.000,1
0.992,1
0.993,1


In [ ]:
# Ваш код

Станок из задачки произвёл по очереди $10$ батчей из деталей. Понятное дело, что со временем со станком могут происходить проблемы, погрешность работы может искажаться, и из-за этого возникает необходимость подкрутить его. 

In [33]:
unique(df$BATCH)

[1]  1  2  3  4  5  6  7  8  9 10

Чтобы понимать, когда подкручивать, на станок нужно настроить какой-то тригер, который будет в случае чего срабатывать и требовать дополнительной проверки станка. Опишите, какую процедуру вы бы для этого использовали. Правда ли, что для $10$ батчей, которые произвёл станок, наш алерт ни разу не сработает? 

In [ ]:
# Ваш код

### [ ] Задачка 6

Эконооооом. [В табличке](https://yadi.sk/d/uT47H5783Xq4jM) `ekanam_data.csv` лежат данные про эконоооооом. [В вспомогательнй табличке](https://yadi.sk/d/MnJnluID3Xq4oL) лежит описание переменных из таблички про эконооооом. Нас будет интересовать не вся таблица, а вполне конкретные столбцы. 

В столбце `year` лежит год набора. В столбце `kozko` лежит массив из нулей и единиц. Единица означает, что лектором по матану был Козко. В столбце `target_1` лежит единичка, если человек дожил до конца первого курса. В столбце `target_2` лежит единица, если человек дожил до конца второго курса. 

In [2]:
df = read.csv('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/2019/ekanam_data.csv', sep=';')
head(df)

year,prohodnoy,lgota,chelevoe,olimp,dogovor,ochko.zaochka,kurs,zima,leto,⋯,X30pub_clust_21,X30pub_clust_22,X30pub_clust_23,X30pub_clust_24,X30pub_clust_25,X30pub_clust_26,X30pub_clust_27,X30pub_clust_28,X30pub_clust_29,pub_likes_cnt
2012,247,0,0,1,0,0,5,0,0,⋯,0.002063675,0.02963645,0.0014059704,0.0021747033,0.09210429,0.009421746,0.005366837,0.08721616,0.0006871135,16
2012,247,0,0,1,0,0,5,0,0,⋯,0.034113219,0.04006099,0.0074222157,0.0019801837,0.02033590,0.005578764,0.006944536,0.04630750,0.0004495845,3
2012,247,0,0,1,0,0,5,0,0,⋯,0.024249456,0.03723640,0.0010818390,0.0041062428,0.09214286,0.006559478,0.018943973,0.04310553,0.0000000000,66
2012,247,1,0,0,0,0,1,0,1,⋯,0.015666752,0.01476522,0.0000000000,0.0000000000,0.10734050,0.001523200,0.043704440,0.01338805,0.0179090897,0
2012,247,1,0,0,0,0,2,1,0,⋯,0.017459158,0.01146120,0.0006298577,0.0006298577,0.00478961,0.017663595,0.021905194,0.05992046,0.0006298577,14
2012,247,1,0,0,0,0,2,1,0,⋯,0.000000000,0.02219519,0.0000000000,0.0000000000,0.03107851,0.006224493,0.017723031,0.11494936,0.0000000000,5


Поговаривают, что Козко тот ещё зверь. Испокон веков, среди студентов эконома ходит миф, что когда матан читает Козко, с первого курса отчисляют больше людей. Когда матан читает Чирский, людей отчисляют меньше. Проверьте гипотезу о том, что при Козко с первого курса отчисляют больше людей. 

In [ ]:
# Ваш код 

Проверьте гипотезу о том, что с первых двух курсов при Козко отчисляют больше людей, чем при Чирском.

In [ ]:
# Ваш код

Сделайте парочку осмысленных и глубоких выводов о том, что вы узнали выше. Как думаете, корректно ли мы проверяем эту гипотезу или что-то мы делаем не так? 

In [ ]:
# Ваши мысли

Подумайте, какие ещё интересные гипотезы можно было бы проверить по экономовскому датасету. Проверьте их. 

In [ ]:
# Ваш код и мысли

### [ ] [Задачка 7](https://nplus1.ru/news/2018/05/11/mice-trolley?utm_source=Telegram&utm_medium=Social) 

Для изучения аспектов процесса принятия моральных решений психологи уже много лет используют этические дилеммы, с помощью которых оценивают действия людей в гипотетических ситуациях. Один из самых известных примеров — это [проблема вагонетки,](https://ru.wikipedia.org/wiki/Проблема_вагонетки) в которой необходимо принять решение о том, стоит ли пожертвовать одним человеком для спасения пятерых.

Бельгийские психологи воплотили дилемму в реальную жизнь. Участники эксперимента должны были выбрать, ударить током пять мышей или одну мышь. Эксперимент проходил следующим образом. Участника сажали перед двумя клетками, в одной из которых сидели пять мышей, а в другой — одна. Перед клетками стоял ноутбук с 20-секундным таймером: участникам сообщили, что по истечении этих 20 секунд в клетку с пятью мышами пустят ток, и мыши получат не смертельный, но очень болезненный удар. Пока время идет, участник может нажать на кнопку: в этом случае ток пустят по клетке с одной мышью. В исследовании использовали живых мышей; следует, однако, уточнить, что удары тока были ненастоящими: сразу же после «удара» участников сопроводили в отдельную комнату, где разъяснили им, что мыши в полном порядке и током их не били (об этом заранее догадались только 12 участников). В решении реальной проблемы вагонетки приняли участие 192 человека, а еще 83 решали такую же задачку, но гипотетическую (на бумаге). Все участники также прошли онлайн-опросы, в ходе которых учёные собрали о респондентах кучу дополнительной информации. 

[В таблице](https://yadi.sk/i/ISGOdtr43Xq7bp) `mouse.csv` лежит информация о том, как прошёл эксперимент. Нас будут интересовать столбцы: 

* STUDY: какую проблему вагонетки решал человек 1 если реальную, 2 если на бумаге
* AGE: возраст респондента 
* GENDER: пол респондента
* DECISION: решение дилеммы (1 - жать на кнопу, 0 - не жать) 
* RT: время, которое респондент потратил, чтобы нажать на кнопку 

Подробное описание данных, сами данные и даже код на R, использованный при оценивании моделей, можно найти в [репозитории исследования.](https://osf.io/kvb99/) В статье авторы строили кучу логистических регрессий, чтобы очистить эффект от психологических особенностей респондентов. Этому добру вас будут учить на эконометрике, мы пойдём простым и не совсем правильным путём и будем проверять гипотезы в лоб. 

In [22]:
df = read.csv('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/2019/mouse.csv', sep='\t')
df = df[c('STUDY', 'AGE', 'GENDER', 'RT')]
head(df)

STUDY,AGE,GENDER,RT
1,38,1,NA
1,22,1,NA
1,39,1,NA
1,21,0,9.212427
1,20,0,17.071711
1,21,1,9.827884


Попытаемся ответить на тот же вопрос, что и исследователи. В какой из двух ситуаций респонденты чащи принимают решение нажать кнопку: в реальной или на бумаге?

In [ ]:
# Ваш код 

Правда ли, что женщины и мужчины принимают решению о нажатии на кнопку одинаково часто? А на бумаге?

In [ ]:
# Ваш код

Правда ли, что женщины намного кровожаднее мужчин и жмут на кнопку гораздо быстрее?

In [ ]:
# Ваш код

Правда ли, что дисперсия времени для женщин равна дисперсии времени для мужчин? 

In [ ]:
# Ваш код

Можем ли мы воспользоваться информацией о равенсве дисперсий для теста на равенство средних? Нужно ли делать коррекцию для уровня значимости? Если да, воспользуйтесь! 

In [ ]:
# Ваш код

Все гипотезы тестируйте на уровне значимости $0.05$, не забывайте выводить $p$-значения. Кстати говоря, обратите внимание, что слово Задачка выделено синим. Это гиперссылка на статью на N+1. 

<img src="мем про веганов" height="300" width="600"> 

### [ ] Задачка 8 

[В датасете]( ) `marijuana-street-price.csv` лежат данные о спросе на марихуанну в США. 

In [34]:
df = read.csv('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/marijuana-street-price.csv', sep=',')
head(df)

State,HighQ,HighQN,MedQ,MedQN,LowQ,LowQN,date
Alabama,339.06,1042,198.64,933,149.49,123,2014-01-01
Alaska,288.75,252,260.60,297,388.58,26,2014-01-01
Arizona,303.31,1941,209.35,1625,189.45,222,2014-01-01
Arkansas,361.85,576,185.62,544,125.87,112,2014-01-01
California,248.78,12096,193.56,12812,192.92,778,2014-01-01
Colorado,236.31,2161,195.29,1728,213.50,128,2014-01-01


### [ ] Задачка 9

28 января 1986 года космический шаттл "Челленджер" взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для 23 предшествовавших катастрофе полётов "Челленджера" известны температура воздуха и появление повреждений хотя бы у одного из уплотнительных колец.

С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после запятой.

Чтобы получить в точности такой же доверительный интервал, как у нас:

* установите `random seed = 0` перед первым вызовом функции `get_bootstrap_samples`,
* один раз сделайте по 1000 псевдовыборок из каждой выборки.


In [ ]:
df = pd.read_csv('challenger.txt',sep='\t')
print df.shape
df.head()

Правда ли, что спрос на траву высокого качества `HighQN` 30 января 2015 года был выше, чем спрос 1 января 2015 года? А правда ли, что он 1 января 2015 года выше, чем 1 января 2014 года? Какое количество наблюдений используется для проверки данной гипотезы? Обоснуйте выбор критерия. 

In [ ]:
# Ваш код

Правда ли, что средняя стоимость травы высокого, среднего и низкого качества в 2015 году совпадает? Обоснуйте выбор критерия, проследите за тем, чтобы уровень значимости не вышел из-под контроля с помощью поправки Бонферонни.

In [ ]:
# Ваш код

Проверьте ту же гипотезу, но используя метод Холма для корекции. 

In [ ]:
# Ваш код

### [ ] Задачка 5

[В табличке](https://yadi.sk/i/_gkQpGru3XabcR) `auto.csv` находятся данные о расходе топлива машинами в галонах на одну милю в США и Японии. Можно ли утверждать, что у Японских двигателей вариация топлива, необходимого для проезда одной мили, меньше чем в США? 

In [ ]:
# Ваш код 

### [ ] Задачка 7

> Помните, что для человека звук его имени является самым сладким и самым важным звуком человеческой речи. (Дейл Карнеги)

Доверяй, но проверяй. Решим задачку по психологии! Все цитаты в пабликах вконтакте делятся на два вида: мудрые и недостаточно мудрые. Если цитата мудрая, то её не грех репостнуть на стеночку. Поговаривают, что если в цитате есть личные местоимения, то пользователь автоматически сопоставляет себя автору этой цитаты и вероятность того, что она буде репостнута повышается. Например, цитату 

> Можно простить человеку все, кроме отсутствия. (Экзюпери) 

мне не очень хотелось бы репостить. А вот цитата 

> Мне наплевать, что вы обо мне думаете. Я о вас вообще не думаю. (Булгаков) 

совершенно другое дело. 

Гипотеза сформулирована. Глаза загорелись. Пришло время проверить её. В [табличке df_vk_wise](https://yadi.sk/i/Dx5hDwkW3XN2Ni) лежат даныне о мудрых цитатах из [мудрого паблика.](https://vk.com/ooioioioioioo) В колонке I находится переменная, которая принимает значение `True`, когда в тексте цитаты было личное местоимение "Я".

Срежем первые $50$ постов, так как под ними количество репостов и лайков ещё окончательно не выставлено и будем работать с остатками. 


In [5]:
df = read.csv('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/df_vk_wise.csv', sep='\t')
df = df[51:nrow(df),]
head(df)

,X,likes,reposts,text,I
51,50,549,25,"Я, говорит, люблю человечество, но дивлюсь на себя самого: чем больше я люблю человечество вообще, тем меньше я люблю людей в частности. Достоевский. Братья Карамазовы",True
52,51,607,22,"Нельзя любить то, что я люблю, да ещё так, как я люблю, и вдобавок жить среди людей. Хулио Кортасар. ""Игра в классики""",True
53,52,914,37,"Если испытываешь по-настоящему глубокое чувство, не стыдись его проявлять. Джон Фаулз. ""Коллекционер""",False
54,53,989,34,"— Я тебя люблю, — сказал я. — Это пройдет, — ответила она и положила трубку. Юрий Визбор",True
55,54,1229,32,"Он очаровательно рассказывает мне о том, как он меня не любит. Марина Цветаева",False
56,55,940,19,"Дорогая, не забывай, что я из Южной Бразилии. Мы по десять лет сохнем по женщине, которую даже не целовали. Элизабет Гилберт, ""Есть, молиться, любить""",True


Проверьте гипотезу о равенстве среднего числа репостов под постами с местоимением "Я" и постами без него. Чётко обоснуйте выбор критерия для этих целей. 

In [ ]:
# Ваш код

Предположим, что число репостов имеет распределение Пуассона с интенсивностью $\lambda$. Проверьте с помощью критерия отношения правдоподобия гипотезу о том, что $\lambda = 100$. 

In [ ]:
# Ваш код

Проверьте ту же гипотезу, используя критерий о равенстве средних. Различаются ли результаты? Как думаете, почему?

In [ ]:
# Ваш код

Проверьте гипотезу о том, что интенсивность репостов под постами с лайками совпадает с интенсивностью под постами без лайков. Используйте для этого критерий отношения правдоподобий. 

In [ ]:
# Ваш код

Если вы помните, у распределение Пуассона $E(X) = Var(X) = \lambda$. Это довольно сильное предположение. Каждый раз, когда мы моделируем какой-то процесс с помощью этого распределения, мы неявно предполагаем наличие такого ограничения. Как бы вы протестировали его наличие для наших данных? Опишите алгоритм и реализуйте его. 

In [ ]:
# Ваш код

### [ ] Задачка 9

Есть ли связь между неграмотностью и рождаемостью? Для $94$ стран, уровень неграмотности женщин в которых больше $5\%$, известны доля неграмотных среди женщин старше $15$ (на $2003$ год) и средняя рождаемость на одну женщину (на $2005$ год). Эти данные лежат [в табличке](https://yadi.sk/i/ae_5JCcZ3Xahbz) `illiteracy.txt`.

In [8]:
df = read.csv('/Users/fulyankin/Yandex.Disk.localized/R/R_prob_data/illiteracy.csv', sep='\t')
head(df)

Country,Illit,Births
Albania,20.5,1.78
Algeria,39.1,2.44
Bahrain,15.0,2.34
Belize,5.9,2.97
Benin,73.5,5.60
Bolivia,18.5,3.65


Построим для этих двух признаков облако рассеивания. 

In [9]:
qplot(Illit, Births, data=df)

ERROR: Error in qplot(Illit, Births, data = df): не могу найти функцию "qplot"


Судя по всему, между переменными есть линейная связь. При росте Illit увеличивается Births. Давайте попробуем посмотреть на величину этой линейной связи. Чему равен выборочный коэффициент корреляции Пирсона между этими двумя признаками?

In [ ]:
# Ваш код 

Давайте проверим гипотезу о равенстве коэффициента корреляции между признаки нулю. Попробуйте использовать для этого критерий, который вам рассказывали на лекции. 

In [ ]:
# Ваш код

Задача со странными вопросами 

- Как распределено pvalue? 

Бутстрэп: 

- Доказать, что в каждой выборке 60% в лекционной частИ!!

- тест выбросов среднее минус медиана бутстрап 

# Кит - рисёрчер врывается на финансовые рынки 

Тут задача с коэфом спирмэна и зависимостью доходнотей от динамики поисковых запросов 